# KoBERT 모델

## 1. 세팅 - mount, install, import, read

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 라이브러리 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install sklearn

     |████████████████████████████████| 46.9 MB 2.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=c232110b5489493d1aeb62109323d93981ec5a8eadc3b0f51f5b02b048cbecb3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 3.9 MB/s 
     |████████████████████████████████| 769 kB 5.5 MB/s 
     |████████████████████████████████| 3.0 MB 53.6 MB/s 
     |████████████████████████████████| 895 kB 75.9 MB/s 


In [ ]:
# 라이브러리 설치 2
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-gvkfto91
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-gvkfto91
     |████████████████████████████████| 4.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 31.2 MB/s 
     |████████████████████████████████| 3.3 MB 47.9 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 61 kB 500 kB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13140 sha256=a6cf3bbef98cea44c75fe564476b226a3d99fb180b598e6592b0d92684ac1c8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ivxfhl_3/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [ ]:
# 라이브러리 임포트
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import joblib

## 2. BERT 세팅

In [ ]:
# 데이터셋 불러오기
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Colab Notebooks/nlp3/dataset/letter_single_train.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/Colab Notebooks/nlp3/dataset/letter_single_train.tsv', field_indices=[0,1], num_discard_samples=1)

# 사랑(0), 좋다(1), 우정(2), 절망(3), 죽음(4), 차분(5), 행복(6), 슬픔(7), 조용(8), 사색(9), 동화(10), 감성(11), 일생(12), 기억(13), 가족(14), 밤(15), 자연(16), 동물(17), 계절(18), 신체(19), 물건(20), 색깔(21), 동네(22), 음식(23), 날씨(24), 여행(25)
print(dataset_train[0])
print(dataset_test[0])

['<br>대한민국 작은 어촌에서 태어난 너! 밥보다 소주가 좋다던 아버지와, 순종적이고 눈물과 희생으로 일구어내신 어머니 사이의 셋째딸로 태어난 너!<br>딸이기 때문에, 형제 중 중간이기 때문에 사랑을 받지 못했고,  아버지, 어머니 하시는 일로 학창시절, 놀림과 따돌림으로 지내왔던 시절이었지!~  그 일로 나는 우울증을 앓았고, 결혼도 할 수 없었지!~<br>지금 와서 생각해보면, 환경도 중요하다는 생각을 하고, 나 자신의 마음가짐, 긍정적인 생각과 좀 더 지식을 알고, 자신감 있게 세상을 헤쳐 나가는 게 얼머나 중요한 지 깨닫는다.<br>아버지다 10년 전 돌아가시고, 어머니와 남동생과 힘든 육체노동을 하며 지내는데, 추운 곳에서 팔  아프고, 허리 아프고, 다리도  아프고, 어깨도 쑤셔가며, 일을 하면서, 남동생이 모질게 대하는 거 참아가며, 이렇게 버텼다.<br>손 마디는 굳은 살이 배겨, 고생을 많이 했다는 걸 알 수 있고, 고달프고, 서글프고, 과로와 스트레스, 짜증이 섞인 하루가 반복되었다.<br>아버지는 왜 우리를 두고, 왜 갑자기 사고(익사사고)로 돌아가셨는지!<br>힘든 가운데, 사람들은 다 떠나가버리고, 배신과 수군거림, 무관심!<br>모든 부정적인 감정과 경험들은 내가 다 경험하는 듯 했다.<br>하지만, 열심히 살고자 하고, 착실하게 자기관리하면서 지내, 도서관에서 강연이 있어, 만나게 된 언니들과 동생 한 명!~<br>그렇게 해서,  성인이 되어 만나게 된 소중한 내 인연!~<br>나에게 용기와 위로를 주고, 심적으로, 또는 물질적으로, 영적으로도 힘이 되어주는 사람들이 있기에,  내가 요즘 힘을 내고 있다. 내가  태어나 이제까지 걸어온 길이 순탄하지는 않았지만, 지금 내 나이 마흔 (불혹)에 와서,<br>조금씩 철이 들고 있다.<br>내가 힘이 들 때, 나 자신에 대해, 부정적으로 생각했고, 생을 다하고 싶다는 생각도 들었지만, 이제 전환점이 온 이때에<br>더욱 자신감을 가지고, 잘 할 수 있다는 믿음과 희망을 가지고, 싶다.

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/root/kobert/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
# 클래스: 데이터셋 → Tensor데이터셋
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 클래스 인자
max_len = 15  # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
# Tensor데이터셋
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

print(data_train[0])
print(data_test[0])

(array([   2,  630,  517,  380,  435,  632, 1683, 3939, 3220, 7449, 6903,
       4720, 6855, 5663,    3], dtype=int32), array(15, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 12)
(array([   2,  630,  517,  380,  435,  632, 1683, 3939, 3220, 7449, 6903,
       4720, 6855, 5663,    3], dtype=int32), array(15, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 12)


In [ ]:
# 배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 3. BERT

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=26, # 클래스 수
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(dataset_train) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.303250789642334 train acc 0.015625
epoch 1 train acc 0.028645833333333332


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.06117424242424243


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 3.1982061862945557 train acc 0.078125
epoch 5 train acc 0.06117424242424243


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 5 test acc 0.10326704545454546


In [ ]:
MODEL_NAME = "kobert_letter_single"
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/nlp3/model/"+ MODEL_NAME

joblib.dump(model, MODEL_PATH + '.pkl')

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        origin_label = [사랑, 좋다, 우정, 절망, 죽음, 차분, 행복, 슬픔, 조용,
                        사색, 동화, 감성, 일생, 기억, 가족, 밤, 자연, 동물,
                        계절, 신체, 물건, 색깔, 동네, 음식, 날씨, 여행]
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            test_eval.append(np.argmax(logits))
        print(">> 입력하신 내용에서 " + origin_label[test_eval[0]] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 0
